In [1]:
# Bring in all dependencies

import pandas as pd
import requests
import numpy as np
import json
import time
from datetime import date
from datetime import timedelta
from pprint import pprint
import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
#from config_poly import api_key
import sqlite3
#from splinter import Browser
#from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
import yfinance as yf

In [3]:
# Establish link to the FINRA OATS list of End of Day List of Stocks
# "http://oatsreportable.finra.org/OATSReportableSecurities-EOD.txt"
stock_list_US = pd.read_csv('nyse_list.csv')

stock_list_US = stock_list_US[['Symbol','Name']]

for i,row in stock_list_US.iterrows():
    if '^' in row['Symbol']:
        stock_list_US.drop(i,inplace=True)
        
# Select 100 random tickers and companies from list of stocks
stock_list_US = stock_list_US.sample(n=100)
stock_list_US
# Creates browser window for scraping
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)

,Symbol,Name
272,AUS,Austerlitz Acquisition Corporation I Class A O...
2236,PICC,Pivotal Investment Corporation III Class A Com...
2802,TPH,Tri Pointe Homes Inc. Common Stock
197,APTS,Preferred Apartment Communities Inc. Common Stock
463,BSA,BrightSphere Investment Group Inc. 5.125% Note...
...,...,...
2718,TARO,Taro Pharmaceutical Industries Ltd. Ordinary S...
1685,LEA,Lear Corporation Common Stock
1115,FINS,Angel Oak Financial Strategies Income Term Tru...
382,BGSF,BGSF Inc. Common Stock


In [4]:
# Testing yahoo finance code
apple = yf.Ticker('AAPL')

# apple.info
# Captures today's date and stores as a string
today = date.today()
yesterday = today - timedelta(days = 1) 
todays_date = today.strftime("%Y-%m-%d")
yesterdays_date = yesterday.strftime("%Y-%m-%d")
# yesterdays_date
apple_last_day = apple.history(period="1mo")
apple_last_day = apple_last_day.reset_index()
apple_last_day['Ticker'] = 'AAPL'

In [9]:
df_list = []
error_log = open("errors.log","a")

startTime = time.time()

# Loops through list of companies on stock_list_US
for _,row in stock_list_US.iterrows():
    
    # Selects ticker from current row
    ticker = yf.Ticker(row['Symbol'])

    # Grabs last opening price, closing price, and dividend amount from the previous day
    # Note: this was done for consistency purposes, to make sure data from the same time and
    # date were being used to create the related K-means clustering model
    try:
        # Grabs ticker history for the defined period
        ticker_history = ticker.history(period="5y")
        # Resets index for ticker history dataframe in order to make date a unique column
        ticker_history = ticker_history.reset_index()
        # Grabs sector information for ticker 
        sector = ticker.info['sector']
        
        # Creates columns in history df to maintain label consistency
        ticker_history['Symbol'] = row['Symbol']
        ticker_history['Name'] = row['Name']
        ticker_history['Sector'] = sector
            
        # Append ticker history dataframe to list of dataframes
        df_list.append(ticker_history)

    # Exception if error comes up 
    except:
        # Writes problem ticker to error log file
        error_log.write(f",{row['Symbol']},\n")
        # Prints out error message for current ticker
        print(f"On {row['Symbol']}, error trying to call")

On BSA, error trying to call
On BVN, error trying to call
On TDI, error trying to call
On SOJD, error trying to call
On FCRZ, error trying to call


In [10]:
# Vertically concatenates (join) the dataframes in df_list into
# one big dataframe
df = pd.concat(df_list)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Name,Sector
0,2021-04-19,9.900000,9.900000,9.900000,9.900000,100.0,0.0,0.0,AUS,Austerlitz Acquisition Corporation I Class A O...,Financial Services
1,2021-04-20,9.945000,10.047000,9.945000,10.040000,1800.0,0.0,0.0,AUS,Austerlitz Acquisition Corporation I Class A O...,Financial Services
2,2021-04-21,9.910000,10.100000,9.900000,9.932000,13200.0,0.0,0.0,AUS,Austerlitz Acquisition Corporation I Class A O...,Financial Services
3,2021-04-22,9.930000,9.930000,9.930000,9.930000,2100.0,0.0,0.0,AUS,Austerlitz Acquisition Corporation I Class A O...,Financial Services
4,2021-04-23,9.930000,9.980000,9.900000,9.930000,43700.0,0.0,0.0,AUS,Austerlitz Acquisition Corporation I Class A O...,Financial Services
...,...,...,...,...,...,...,...,...,...,...,...
76,2021-05-21,24.350000,24.350000,24.350000,24.350000,0.0,0.0,0.0,PCPC,Periphas Capital Partnering Corporation Class ...,Financial Services
77,2021-05-24,24.350000,24.350000,24.350000,24.350000,0.0,0.0,0.0,PCPC,Periphas Capital Partnering Corporation Class ...,Financial Services
78,2021-05-25,24.400000,24.400000,24.389999,24.389999,1600.0,0.0,0.0,PCPC,Periphas Capital Partnering Corporation Class ...,Financial Services
79,2021-05-26,24.750000,24.750000,24.389999,24.389999,10100.0,0.0,0.0,PCPC,Periphas Capital Partnering Corporation Class ...,Financial Services


In [11]:
df.to_csv('model_data.csv')

In [4]:

# # Creates empty lists to save tickers, company names,
# # and stock exchange information

# tickers = []
# name = []
# exchange = []

# # Loops through stocks list and fills each empty list
# for s in stocks:
#     info = s.split("|")
#     tickers.append(info[0])
#     name.append(info[1])
#     exchange.append(info[2])

In [5]:
# # Creates dictionary with ticker symbol, company name,
# # and stock exchange information
# info_dict = {'Ticker Symbol':tickers,'Company':name,'Exchange':exchange}

# # Creates new pandas dataframe containing dictionary information
# info_df = pd.DataFrame(info_dict)

# tickers = info_df['Ticker Symbol']
# tickers

0            A
1           AA
2          AAA
3        AAAIF
4        AAALF
         ...  
27590     ZYXI
27591    ZZHGF
27592    ZZHGY
27593     ZZLL
27594    ZZZOD
Name: Ticker Symbol, Length: 27595, dtype: object

In [6]:
# Update: the code below has been commented out
# due to the use of a new Polygon.io subscription
# which eliminates the need for a sleep function
# New for loop posted below

In [7]:
# # Initialize list to hold information from polygon.io
# polygon_info = []
# polygon_div = []

# startTime = time.time()
# #today = date.today()
# #todays_date = today.strftime("%Y-%m-%d")

# # Initializes counter to 0
# count = 0

# error_log = open("errors.log","a")
    
# # Loops through one list of ticker symbols
# for ticker in tickers:

#     # Checks if count equals 4, used to prevent exceeding 
#     # the API call limit of 5 calls/minute
    
#     # Update: with new subscription, no if statement needed
#     # If statement to be commented out
#     #if count == 4:

#         # Tells program to wait 1.5 minutes before running again
#         #time.sleep(60)

#         # Ticker symbols to be used for API Call
#         #dataset_code = ticker

#         # URLs for API Calls, the first for open and closed prices, 
#         # the second for divident information
#         #url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
#         #div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#         # Saving responses from both urls
#         #try:
#         #    response = requests.get(url).json()
#         #    div_response = requests.get(div_url).json()

#             # Adding responses to info and div lists, respectively
#         #    polygon_info.append(response)
#             polygon_div.append(div_response)

#         except:
#             error_log.write(ticker)
#             print(f"On {ticker}, error trying to call")

#         # Resets counter
#         count = 0

#     else:
#         # Ticker symbols to be used for API Call
#         dataset_code = ticker

#         # URLs for API Calls, the first for open and closed prices, 
#         # the second for divident information
#         url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
#         div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#          # Saving responses from both urls
#         try:
#             response = requests.get(url).json()
#             div_response = requests.get(div_url).json()

#             # Adding responses to info and div lists, respectively
#             polygon_info.append(response)
#             polygon_div.append(div_response)

#         except:
#             error_log.write(ticker)
#             print(f"On {ticker}, error trying to call")

#         # Increase counter by 2
#         count += 2



In [9]:
# Initialize lists to hold information from polygon.io
polygon_info = []
polygon_div = []

error_log = open("errors.log","a")
bad_ticker_log = open("bad_ticker.log","a")

count = 1

# Captures start time for loop
startTime = time.time()

for ticker in tickers:
# Ticker symbols to be used for API Call
    
    if count > 5:
        
        time.sleep(65)
        
        dataset_code = ticker

        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/{todays_date}?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

         # Saving responses from both urls
        try:
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            if response['status'] == 'OK':
            # Adding responses to info and div lists, respectively
                polygon_info.append(response)
                polygon_div.append(div_response)
                
            else:
                bad_ticker_log.write(ticker)

        except:
            error_log.write(ticker)
            print(f"On {ticker}, error trying to call")
            
        count = 1
        
    else:
        
        dataset_code = ticker

        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/{todays_date}?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

         # Saving responses from both urls
        try:
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            if response['status'] == 'OK':
            # Adding responses to info and div lists, respectively
                polygon_info.append(response)
                polygon_div.append(div_response)
                
            else:
                bad_ticker_log.write(ticker)

        except:
            error_log.write(ticker)
            print(f"On {ticker}, error trying to call")
            
        count += 1
            
executionTime = (time.time() - startTime)

On BLRDF, error trying to call


KeyboardInterrupt: 

In [ ]:
realTime = executionTime/60
realTime_hr = realTime/60
realTime_hr

In [ ]:
len(polygon_info)

In [ ]:
polygon_info_2 = []
polygon_div_2 = []

read_error = open("errors.log","r")
startTime = time.time()

# Loops through one list of ticker symbols
for ticker in read_error:
    # Ticker symbols to be used for API Call
    dataset_code = ticker

    # URLs for API Calls, the first for open and closed prices, 
    # the second for divident information
    url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
    div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

    # Saving responses from both urls
    try:
        response = requests.get(url).json()
        div_response = requests.get(div_url).json()

        # Adding responses to info and div lists, respectively
        polygon_info_2.append(response)
        polygon_div_2.append(div_response)

    except:
        print(f"On {ticker}, error trying to call")

In [ ]:
executionTime = (time.time() - startTime)

realTime = executionTime/60
realTime

In [10]:
real_info = []

for t in polygon_info:
    if t['status'] == 'OK':
        real_info.append(t)
        
len(real_info)

1243

In [12]:
real_info_df = pd.DataFrame(real_info)
real_info_df = real_info_df[['symbol','from','open','high','low','close','volume']]
#polygon_info_df.columns = ['symbol','date','open','high','low','close','volume']
real_info_df

,symbol,from,open,high,low,close,volume
0,A,2021-05-25,133.410,134.800,133.010,133.2300,1905708
1,AA,2021-05-25,36.460,36.930,34.940,35.1500,5941039
2,AAA,2021-05-25,25.001,25.020,25.000,25.0200,9286
3,AAAU,2021-05-25,18.730,18.910,18.710,18.9100,522160
4,AAC,2021-05-25,9.800,9.860,9.790,9.8100,59427
...,...,...,...,...,...,...,...
1238,BUD,2021-05-25,76.100,76.150,75.582,75.6600,830950
1239,BUFD,2021-05-25,20.630,20.650,20.570,20.6347,57116
1240,BUFF,2021-05-25,34.550,34.578,34.460,34.5144,13116
1241,BUFR,2021-05-25,22.720,22.720,22.600,22.6000,75602


In [ ]:
all_div = polygon_div[1:]
all_div

# len(all_div)

# for div in all_div:
#     print(div['results'])
#     print()

In [ ]:
real_div = []

for t in polygon_div[1:]:
    if t['status'] == 'OK' and t['count'] > 0:
        real_div.append(t)

In [ ]:
#all_div = polygon_div[1:]
div_amt_list = []
payDate = []
ticker_list = []

for div in real_div:
    result = div['results']
    
    if result == []:
        continue
        
    else:
        ticker = div['results'][0]['ticker']
        div_amt = div['results'][0]['amount']
        pay_date = div["results"][0]['paymentDate']
    
    ticker_list.append(ticker)
    div_amt_list.append(div_amt)
    payDate.append(pay_date)
    
polygon_div_df = pd.DataFrame({"Symbol":ticker_list,"Amount ($)":div_amt_list,"Payment Date":payDate})

In [ ]:
polygon_div_df

In [ ]:
polygon_info_df.to_csv('openClose.csv')
polygon_div_df.to_csv('dividend.csv')
company_table.to_csv('companies.csv')

In [ ]:
polygon_info_df = pd.read_csv('openClose.csv')
polygon_div_df = pd.read_csv('dividend.csv')
company_table = pd.read_csv('companies.csv')

polygon_info_df.columns = ['index','symbol','date','opening','high','low','close','volume']
polygon_info_df = polygon_info_df.drop(columns='index')
polygon_div_df

In [ ]:
del polygon_div_df['Unnamed: 0']

In [ ]:
del company_table['Unnamed: 0']
company_table

In [ ]:
# Create sqlite database path
database_path = "russell_stock.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [ ]:
# use engine to connect to existing tables/db
Base = declarative_base()
#conn = sqlite3.connect('russell_stock.sqlite')
#c = conn.cursor()

# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(bind=engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
#Base.metadata.create_all(engine)

In [ ]:
# Define our General_Info table
class Open_Close(Base):
    __tablename__ = 'openClose'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    date = Column(String)
    opening = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Integer)

# Define our Dividend table
class Dividend(Base):
    __tablename__ = 'dividend'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    amount = Column(Float)
    payment_date = Column(String)

# Define our Company Table
class Company(Base):
    __tablename__ = 'company'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    name = Column(String)

In [ ]:
# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
#Open_Close = Database.classes.open_close

In [ ]:
#for i in polygon_info_df.index:
#    print(polygon_info_df['from'][i])


# insert dataframe into sqlite tables
company_table.to_sql(name='company', con=engine, if_exists='replace', index=True)
polygon_div_df.to_sql(name='dividend', con=engine, if_exists='replace', index=True)
polygon_info_df.to_sql(name='openClose', con=engine, if_exists='replace', index=True)

In [ ]:
# # for i in polygon_info_df.index:
#     thisFrom = polygon_info_df["date"][i]
#     thisSymbol = polygon_info_df["symbol"][i]
#     thisOpen = polygon_info_df["open"][i]
#     thisHigh = polygon_info_df["high"][i]
#    thisLow = polygon_info_df["low"][i]
#    thisClose = polygon_info_df["close"][i]
#    thisVol = polygon_info_df["volume"][i]
    
    #row = [thisFrom,thisSymbol,thisOpen,thisHigh,thisLow,thisClose,thisVol]
    
#    thisCompany = Open_Close(date=thisFrom,symbol=thisSymbol,opening=thisOpen,high=thisHigh,low=thisLow,close=thisClose,volume=thisVol)
#    session = Session(bind=engine)
    #conn.execute('Insert into open_close values(?,?,?,?,?,?,?)', row)
    #engine.execute()
#    Session.configure(bind=engine)
#    session.add(thisCompany)
    #session.commit()
    
#session.close()
    
    
    
    

In [ ]:
#c.close()
#session.commit()



In [ ]:
#session.close()

#conn.close()

engine.dispose()

In [ ]:
engine = create_engine(f"sqlite:///{database_path}")

connection = engine.connect()

# use engine to connect to existing tables/db
base = automap_base()
base.prepare(engine, reflect=True)

# View all of the classes/tables that automap found
thing = base.classes.keys()

print(thing)

# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
inspector = inspect(engine)

In [ ]:
connection.close()

engine.dispose()

In [7]:
import os

for i in range(0,2000,100):
    start = i
    stop =  i + 100
    script_text = '''{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Untitled0.ipynb",
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "32ADhwU0Vgmv",
        "outputId": "2a4e86e5-504a-4a0b-a2ff-c709da94d81e"
      },
      "source": [
'''
    script_text_2 = f'''        "!pip install yfinance",
        "!git clone https://github.com/emjasmine/TeamMoneyPenny-ML.git",
        "%cd TeamMoneyPenny-ML",
        "!pwd",
        "!python yfinance_API_client_end.py {start} {stop}",
        "!ls",
        "!git add .",
        "!git commit -m \"Adding new files from {start} to {stop}\"",
        "!git push"'''
    script_text_3 = '''
      ],
      "execution_count": 19,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "0-10_stock_info.csv  nasdaq_screener.csv  templates",
            "0-2_stock_info.csv   nasdaq_screener.xls  US-Stock-Symbols.csv",
            "0-3_stock_info.csv   nyse_list.csv\t  US-Stock-Symbols.xlsx",
            "0-5_stock_info.csv   nyse_list.xls\t  yfinance_API_client_end.py",
            "API_Calls.ipynb      README.md",
            "errors.log\t     static"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "BiTqAdc4WLCn"
      },
      "source": [
        "# New Section"
      ]
    }
  ]
}
'''
    path = os.path.join('Colabs',f'colab_{start}_{stop}.ipynb')
    file = open(path,'w')
    file.write(script_text+script_text_2+script_text_3)
    file.close()


In [6]:
!ls

0-10_stock_info.csv        Untitled0.ipynb
0-2_stock_info.csv         errors.log
0-5_stock_info.csv         nasdaq_screener.csv
API_Calls.ipynb            nasdaq_screener.xls
Colabs                     nyse_list.csv
Data_Sources               nyse_list.xls
README.md                  static
US-Stock-Symbols.csv       templates
US-Stock-Symbols.xlsx      yfinance_API_client_end.py
